In [1]:
using ITensors

# To Do 

- Add comments explaining stuff
- combine swap and beamsplitter into one function
- Maybe not destroy the original MPO when taking a projection ✅
- Finally actually check if we are getting the excpected state (bell state of sorts)
- write out measuring the stabalisers
- think about how to add noise cavities


# Functions

In [884]:
function ITensors.op(::OpName"Ground" , ::SiteType"Qudit" , d::Int)
    mat = zeros(d, d)
    mat[1,1] = 1
    return mat
end

In [885]:
function ITensors.op(::OpName"Excite1" , ::SiteType"Qudit" , d::Int)
    mat = zeros(d, d)
    mat[2,2] = 1
    return mat
end

In [886]:
function ITensors.expect(MPO_i, sites_i, N)
    list_exp = []
    for i in 1:N
        append!(list_exp, tr(apply( op("N", sites_i[i]), MPO_i)))
    end
    return list_exp
end

In [887]:
function beamsplitter(MPO_i, index_1, index_2, site_list)

    op_1 = ((op("A",site_list[index_1]) * op("Adag",site_list[index_2])) + (op("A",site_list[index_2]) * op("Adag",site_list[index_1])))
    H_ = exp((-im/4) * pi * op_1)

    H2 = apply(H_, MPO_i ; cutoff)
    H3 = conj(swapprime( apply( H_, swapprime(conj(H2), 1,0); cutoff ), 1,0))

    MPO_i[:] = H3
    return nothing
end

beamsplitter (generic function with 1 method)

In [888]:
function swap(MPO_i, index_1, index_2, site_list)

    op_1 = ((op("A",site_list[index_1]) * op("Adag",site_list[index_2])) + (op("A",site_list[index_2]) * op("Adag",site_list[index_1])))
    H_ = exp((-im/2) * pi * op_1)

    H2 = apply(H_, MPO_i ; cutoff, maxdim)
    H3 = conj(swapprime( apply( H_, swapprime(conj(H2), 1,0); cutoff, maxdim ), 1,0))

    MPO_i[:] = H3
    return nothing
end

swap (generic function with 1 method)

# Main Code

In [889]:
N = 8
sites = siteinds("Qudit", N, dim=3)
cutoff = 1E-8
maxdim = 100

100

In [890]:
input_list = [  repeat(["Excite1",], trunc(Int, N/2)) ; repeat(["Excite1",], trunc(Int, N/2)) ]


8-element Vector{String}:
 "Excite1"
 "Excite1"
 "Excite1"
 "Excite1"
 "Excite1"
 "Excite1"
 "Excite1"
 "Excite1"

In [891]:
rails = MPO(sites, input_list)
excited_rail = MPO(sites, "Ground")

MPO
[1] ((dim=3|id=53|"Qudit,Site,n=1")', (dim=3|id=53|"Qudit,Site,n=1"), (dim=1|id=250|"Link,l=1"))
[2] ((dim=3|id=316|"Qudit,Site,n=2")', (dim=3|id=316|"Qudit,Site,n=2"), (dim=1|id=649|"Link,l=2"), (dim=1|id=250|"Link,l=1"))
[3] ((dim=3|id=245|"Qudit,Site,n=3")', (dim=3|id=245|"Qudit,Site,n=3"), (dim=1|id=107|"Link,l=3"), (dim=1|id=649|"Link,l=2"))
[4] ((dim=3|id=883|"Qudit,Site,n=4")', (dim=3|id=883|"Qudit,Site,n=4"), (dim=1|id=629|"Link,l=4"), (dim=1|id=107|"Link,l=3"))
[5] ((dim=3|id=919|"Qudit,Site,n=5")', (dim=3|id=919|"Qudit,Site,n=5"), (dim=1|id=961|"Link,l=5"), (dim=1|id=629|"Link,l=4"))
[6] ((dim=3|id=269|"Qudit,Site,n=6")', (dim=3|id=269|"Qudit,Site,n=6"), (dim=1|id=481|"Link,l=6"), (dim=1|id=961|"Link,l=5"))
[7] ((dim=3|id=944|"Qudit,Site,n=7")', (dim=3|id=944|"Qudit,Site,n=7"), (dim=1|id=493|"Link,l=7"), (dim=1|id=481|"Link,l=6"))
[8] ((dim=3|id=830|"Qudit,Site,n=8")', (dim=3|id=830|"Qudit,Site,n=8"), (dim=1|id=493|"Link,l=7"))


In [892]:
expect(rails, sites)

8-element Vector{Any}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [782]:
beamsplitter(rails, 2, 6, sites)
normalize!(rails)
println(tr(rails))

1.0 + 0.0im


In [783]:

beamsplitter(rails, 3, 7, sites)
normalize!(rails)
println(tr(rails))
beamsplitter(rails, 4, 8, sites)
normalize!(rails)
println(tr(rails))
beamsplitter(rails, 1, 5, sites)
normalize!(rails)
println(tr(rails))
#cutoff = 20


0.9999999999999998 + 1.8169632348103397e-16im
0.9999999999999982

 - 1.3671525712072973e-16im


0.9999999999999978 + 1.5006352874974966e-16im


In [784]:
beamsplitter(rails, 5, 6, sites)
normalize!(rails)
println(tr(rails))
beamsplitter(rails, 7, 8, sites)
normalize!(rails)
println(tr(rails))
beamsplitter(rails, 5, 8, sites)
normalize!(rails)
println(tr(rails))
beamsplitter(rails, 6, 7, sites)
normalize!(rails)
println(tr(rails))

0.9999999999999961 + 5.173483756631972e-16im
0.9999999999999967

 + 2.089064435542515e-16im


0.9999999999999977 - 6.908616133495565e-16im
0.9999999999999977

 - 6.848952571757802e-16im


In [785]:
tr(rails)

0.9999999999999977 - 6.848952571757802e-16im

In [786]:
swap(rails, 6, 8, sites)

In [787]:
tr(rails)

0.9999999999999958 - 7.601258468329633e-16im

In [788]:
expect(rails, sites)/tr(rails)

8-element Vector{ComplexF64}:
 0.49999999999999967 + 2.0887248224420326e-15im
  0.5000000000000019 + 8.340857101884456e-17im
  0.5000000000000001 - 1.7808501107323514e-15im
  0.4999999999999994 + 1.204489431412828e-16im
  0.5000000000000002 - 6.095686331612628e-16im
  0.4999999999999999 + 2.73231719765038e-16im
  0.4999999999999993 - 2.5189295416095423e-16im
  0.5000000000000021 - 1.8884575824153157e-15im

# Checking BS works

In [893]:
el = [2,2,2,2]

V = ITensor(1.)
for j=8:-1:5
  V *= ( state(sites[j],el[j-4])) * prime(state(sites[j],el[j-4])) * rails[j]
end

tensor = rails[1]*rails[2]*rails[3] *rails[4] * V
sub_sites = sites[1:4]


4-element Vector{Index{Int64}}:
 (dim=3|id=53|"Qudit,Site,n=1")
 (dim=3|id=316|"Qudit,Site,n=2")
 (dim=3|id=245|"Qudit,Site,n=3")
 (dim=3|id=883|"Qudit,Site,n=4")

In [896]:
tr(MPO( tensor ,sites[1:4]))

1.0

In [897]:
expect(MPO( tensor ,sites[1:4]),sub_sites, 4)

4-element Vector{Any}:
 1.0
 1.0
 1.0
 1.0

# Appendix

function beamsplitter(MPO_i, index_1, index_2, site_list)
    #op_1 = op("Id",site_list[1])
    #op_1 = sqrt(0.5)* ((op("A",site_list[index_1]) * op("Adag",site_list[index_2])) + (op("A",site_list[index_2]) * op("Adag",site_list[index_1])))
    op_1 = ((op("A",site_list[index_1]) * op("Adag",site_list[index_2])) + (op("A",site_list[index_2]) * op("Adag",site_list[index_1])))
    #op_1 *= op("Id", site_list[3])
    #op_1 *= op("Id", site_list[2])

    #for i in 2:length(site_list)
    #    if i == index_1 || i==index_2
    #        nothing
    #    else
    #        op_1 *= op("Id",site_list[i])
    #    end
    #end
    #op_2 = op("A",site_list[index_2]) * op("Adag",site_list[index_1])
    #for i in 1:length(site_list)
    #    if i == index_1 || i==index_2
    #        nothing
    #    else
    #        op_2 *= op("Id",site_list[i])
    #    end
    #end
    #os = op_1 + op_2
    H_bs = exp((-im/4) * pi * op_1)
    #term2 = MPO(H_bs, site_list)

    c = cutoff
    d = maxdim

    #MPO_i[:] = apply(apply(term2, MPO_i ; cutoff= c , maxdim = d ),term2 ; cutoff= c , maxdim= d)
    return op_1
end


#Notes; 
#1. Product still works with one common index
#1.1 Order of MPO/A matters
#2. Apply needs common index with zero prime at least
#3. A(0,1) and rails(0,x) works
#4. A(0,x) and rails(0,x) or A(0,x) and rails(0,1) dont work

#Apply is complicated

In [804]:
A = randomITensor(sites[5:8]) * 0
A[1, 1] = 1
rails_MPS = MPS(A ,sites[5:8] ; cutoff = cutoff, maxdim = maxdim )

A[1, 1] = 1.0


1.0

In [801]:
Nvals = expect(rails_MPS,"N")

8-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0